## Lettura file


### Lettura del file assets_dataset.csv


In [204]:
File_Assets=open("assets_dataset.csv", "r")
File_Cov=open("./assets_cov_matrix.csv", "r")
AssVet = []
for count, i in enumerate(File_Assets):
    if count!=0:
        id=i.split("_")[1].split(",")[0]
        num1=i.split(",")[1].strip()
        num2=i.split(",")[2].strip()
        AssVet.append([float(num1)*10,float(num2)*10])

### Lettura del file assets_cov_matrix.csv


In [205]:
#Lettura del file assets_cov_matrix.csv
MatCov=[]
n=[]
for count, i in enumerate(File_Cov):
    if count!=0:
        n=i.split(",")
        for j in range(500-1):
            n[j]=float(n[j+1])
        MatCov.append(n)

## Conversione delle variabili float in vettore di variabili binarie

In [206]:
def Discretization(p, prec, arr):
    BinAssVet = [] #scomposizione binaria
    for i in arr:
        bin = []
        num = i
        k = p-1
        while k>=0:
            CurrVal = ((2 ** k) / prec) #
            if num > CurrVal:
                num = num - CurrVal
                bin.append(1)
            else:
                bin.append(0)
            k=k-1
        ConvVal = 0
        for j in range(len(bin)):
            ConvVal = ConvVal + bin[p-j-1]*((2 ** j) / prec)
        print(f"{i}  {bin} errore {(i-ConvVal)}")
        ##print(f"{bin}")
        BinAssVet.append(bin)
    #print(arr)
Discretization(12,10, [i[0] for i in AssVet])
Discretization(12,10, [i[1] for i in AssVet])

0.8323991651469052  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] errore 0.03239916514690511
1.1619757425426565  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1] errore 0.061975742542656365
1.0406505058608655  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0] errore 0.04065050586086549
1.2489679762576336  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] errore 0.04896797625763338
0.7168007644943526  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1] errore 0.016800764494352527
0.7217838741608622  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1] errore 0.021783874160862138
0.7081008680559406  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1] errore 0.008100868055940569
0.7336048380629118  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1] errore 0.0336048380629117
1.02386870677326  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0] errore 0.023868706773259918
0.7570345401248442  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1] errore 0.0570345401248441
1.3817771686923919  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1] errore 0.08177716869239182
1.446389009609816  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0] errore 0.0463890096

## Proposed formulation [3.2]

In [207]:

from qubovert import boolean_var, QUBO
from qubovert.sim import anneal_qubo
import numpy as np


#Creazione delle variabili booleane
#x_i=1 se l'asset i è presente nel portafoglio, x_i=0 altrimenti
def create_boolean_variables(n_assets):
    # Creazione della matrice delle variabili booleane, inizializzate a None
    var_matrix = [[None for j in range(n_assets)] for i in range(n_assets)]
    for i in range(n_assets):
        for j in range (n_assets):
            var =boolean_var(f"x_{i}{j}")	
            var_matrix[i][j]=var
    return var_matrix


# creazione della funzione obiettivo per H0
def create_H0(n_assets, precision, MatCov, coeff, x):
    model_H0= QUBO()
    for i in range(n_assets):
        for j in range(i, n_assets):
            for k in range(precision-1):
                for l in range(precision-1):
                    model_H0+=MatCov[i][j]*(coeff[k]*x[i][k])*(coeff[l]*x[j][l])
    return model_H0
# creazione della funzione obiettivo per H1
def create_H1(n_assets, precision, mu, coeff, x):
    model_H1= QUBO()
    for i in range(n_assets):
        for j in range(precision-1):
            model_H1+=(mu[i]*coeff[j]*x[i][j]-1)**2
    return model_H1
l0 = 0.005
l1 = 0.1

n_assets = 3
precision = 3

mu=[0.1,0.3, 0.7]
coeff=[2**(-i) for i in range(precision)]

model=QUBO()

x = create_boolean_variables(n_assets)

MatCov = np.array([
    [0.00044494, 0.00040597, 0.00086485],
    [0.00040597, 0.00170461, 0.00170597],
    [0.00086485, 0.00170597, 0.00437225]    
])

model_H0 = create_H0(n_assets, precision, MatCov, coeff,x)
model_H1 = create_H1(n_assets, precision, mu, coeff,x)
model = -(l0*model_H0 + l1*model_H1)

solution =anneal_qubo(model,  num_anneals=10)




print(solution)


AnnealResults

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0, 'x_10': 0, 'x_11': 0, 'x_20': 0, 'x_21': 0}
  value: -0.6
   spin: False

  state: {'x_00': 0, 'x_01': 0,

## Formulation for diversified portfolio Optimization

In [208]:
model_H2 = QbVrt.QUBO()
temp= QbVrt.QUBO()
f = [] #da definire
D = [] #da definire
# creazione della funzione obiettivo per H2
def create_H2(n_assets, precision, model_H2, f, D):
    temp= QbVrt.QUBO()
    temp2= QbVrt.QUBO()
    for i in range(n_assets):
        for j in range(precision-1):
            temp+=f[i]*x[i][j]
    
    for i in range(n_assets):
        for k in range(precision-1):
            for j in range(n_assets):
                for l in range(precision-1):
                    temp2+=D[i][j]*x[i][k]*x[j][l]
    model_H2=-temp-temp2
    return model_H2

model = l0*model_H0 + l1*model_H1 + l2*model_H2
